# Arbre enraciné de profit maximum avec des contraintes de capacité

Etant donné un ensemble de sites $V$ et une matrice de distance $D = (d_{ij})$ entre chaque paire de sites, le problème du $p$-centre consiste à sélectionner $p$ sites de $V$ afin qu'ils agissent comme des centres et à affecter les autres sites au centre disponible le plus proche tout en minimisant la plus grande distance entre un site et son centre. Des contraintes supplémentaires, telles que des contraintes de précédence ou des contraintes de capacité, peuvent apparaître dans certaines applications. Ces problèmes sont souvent résolus par des méthodes de décomposition où le problème original est décomposé en sous-problèmes consistant à construire des centres réalisables.

Le but de ce projet est de proposer des modèles mathématiques pour la résolution d'un sous-problème associé à un site donné jouant le rôle de centre.

## Présentation du problème

Soient $V = \{0, \dots, n-1\}$ un ensemble de sites et $r \in V$ un centre prédéfini. A chaque site $i \in V$, on associe deux poids $w_i^1$ et $w_i^2$, une distance $d_i$ entre le site $i$ et le centre $r$, et un profit $p_i$ (pouvant être négatif). On définit également pour chaque site $i$ un prédécesseur $\pi_i \in V$ devant être pris si $i$ est pris dans la solution. On suppose que les données satisfont la propriété que la distance au centre d'un sommet $i$ est supérieure ou égale à la distance au centre de son précédesseur $\pi_i$. L'ensemble des prédécesseurs défini un arbre enraciné en le centre $r$. Soient $C^1$ et $C^2$ deux capacités.

Le problème consiste à selectionner un ensemble de sommets qui seront affectés au centre $r$, dont le poids total (centre inclu), pour chacun des poids $w^1$ et $w^2$ est inférieur aux capacités $C^1$ et $C^2$, et tel que si un sommet $i$ est sélectionné, alors sont prédécesseur $\pi_i$ est également sélectionné.

L'objectif du problème est de maximiser le profit total des sites sélectionnés (en incluant le centre $r$) auquel on soustrait la distance au centre (également appelée rayon) du sommet sélectionné le plus éloigné.


## Données

Vous trouverez dans le répertoire `instances` un ensemble de jeux de données pour ce problème. Chaque fichier correspond à un jeu de données et contient les informations suivantes (la numérotation des sites commence à 0) :

- Première ligne : Nombre de sites, $C^1$, $C^2$, numéro du centre.
- Une ligne pour chaque site indiquant : numéro du prédécesseur $\pi_i$, distance $d_i$ du site au centre, le profit $p_i$ du site, le poids $w_i^1$, le poids $w_i^2$.

Si pour un site $i$, son prédécesseur vaut $\pi_i = -1$, soit $i$ est le centre $r$ et il doit être pris dans la solution, soit le site $i$ n'est pas le centre et auquel cas, le site ne peut pas être pris dans la solution.

Implémentez ici la lecture des données.

In [1]:
# Instances de test

#dataFileName = 'Instances/btk_1-Reichstett_0_0.dat'
#modelFileName = 'Models/btk_1-Reichstett_0_0.lp'
#soluFileName = 'Solutions/btk_1-Reichstett_0_0.lp'

In [2]:
import os

# get the list of filenames in the working directory

#os.chdir("/media/pachauveau/SanDisk/ROAD/IPVE/Projet/")
os.chdir("/media/paul/SanDisk/ROAD/IPVE/Projet")
#print(os.getcwd())
os.chdir("./Instances")
#print(os.getcwd())

namesAndExtensions = os.listdir('.')
# print(namesAndExtensions)
## ['btk_att48_2_3_2_4.dat', 'btk_eilA76_3_2_2_0.dat', ... ]

# splittedStr = filter(split('.'), namesAndExtensions)

names = []
extensions = []

# si il y a un point dans le nom du fichier on récupère 
# alors seulement une partie des valeurs
for nAndE in namesAndExtensions :
    splittedStr = nAndE.split('.')
    extensions.append("." + splittedStr[-1])
    names.append(splittedStr[-2])

print(names[0] + " - " + extensions[0])


os.chdir("..")
print(os.getcwd())

btk_att48_2_3_2_4 - .dat
/media/paul/SanDisk/ROAD/IPVE/Projet


In [3]:
def putDatExt(str):
    return (str + ".dat")

def putIns(str):
    return ("Instances/" + str)

namesWithIns = list(map(putIns, names))

namesWithInsAndDat = list(map(putDatExt, namesWithIns))

# print(namesWithInsAndDat[0])

In [4]:
def putLpExt(str):
    return str + ".lp"

def putMod(str):
    return "Models/" + str

namesWithMod = list(map(putMod, names))

namesWithModAndLp = list(map(putLpExt, namesWithMod))

#print(namesWithModAndLp[0])

In [5]:
def putSol(str):
    return "Solutions/" + str

namesWithSol = list(map(putSol, names))

namesWithSolAndLp = list(map(putLpExt, namesWithSol))

#print(namesWithSolAndLp[0])

In [6]:
with open(namesWithInsAndDat[0], "r") as file:
    
    # première ligne
    line = file.readline()
    lineTab = line.split()
    
    n = int(lineTab[0]) # nombre de sites
    C_1 = int(lineTab[1]) # capacité max 1
    C_2 = int(lineTab[2]) # capacité max 2
    r = int(lineTab[3]) # numéro du centre
    
    # print(n)
    
    PI = [] # prédécesseurs
    d = [] # distances
    p = [] # profits
    w_1 = [] # poids 1
    w_2 = [] # poids 2
    
    
    for j in range(n): # pour chaque site
        line = file.readline()
        lineTab = line.split()
        
        # print(j)
        
        PI.append(int(lineTab[0]))
        d.append(int(lineTab[1]))
        p.append(float(lineTab[2]))
        w_1.append(int(lineTab[3]))
        w_2.append(int(lineTab[4]))

## Premier modèle

Considérez les variables :

- $x_i = \left\{ \begin{array}{l}
1 \text{ si le site } i \in V \text{ est sélectionné,}\\
0 \text{ sinon.}
\end{array}\right.$
- $R \in \mathbb{R} $ la valeur du rayon (la distance au centre du sommet sélectionné le plus éloigné).

Ecrivez et implémentez ce premier modèle. Vous testerez ce modèle sur les instances fournies.

In [7]:
# Import du paquet PythonMIP et de toutes ses fonctionnalités
from mip import *

# Import du paquet time pour calculer le temps de résolution
import time

# Création du modèle vide 
model1 = Model(name = "RSWCC1", solver_name="CBC")
# model1 = Model(name = "RSWCC1")

## Variables

# x(i)
x = []
for i in range(n):
    x.append(model1.add_var(name="x(" + str(i) + ")", var_type=BINARY))
             
# R
R = model1.add_var(name="R", var_type=CONTINUOUS)
             
## Objectif

model1.objective = maximize(xsum(x[i]*p[i] for i in range(n)) - R)

In [8]:
## Contraintes

# Respect de la capacité 1 
model1.add_constr(xsum(x[i]*w_1[i] for i in range(n)) <= C_1)

# Respect de la capacité 2
model1.add_constr(xsum(x[i]*w_2[i] for i in range(n)) <= C_2)

# Tout sommet à un prédécesseur
for i in range(n):
    if (d[i] != 0):
        model1.add_constr(x[i] <= x[PI[i]])
    else:
        model1.add_constr(x[i] == -1)
        
# Ci dessus, on considère que les données sont bien construites
# sinon on devrait générer des exceptions diverses
    
# La distance R est la plus grande distance séléctionnée
for i in range(n):
    model1.add_constr(x[i]*d[i] <= R)

In [9]:
model1.write(namesWithModAndLp[0])
    
start = time.perf_counter()

status = model1.optimize(max_seconds = 60)

runtime = time.perf_counter() - start

Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 15 2020 

Starting solution of the Linear programming relaxation problem using Primal Simplex



In [10]:
with open(namesWithSolAndLp[0], 'w') as file:  #ouvre le fichier, le ferme automatiquement à la fin et gère les exceptions
    def dprint(str): # double print (file and console)
        print(str)
        file.write(str + "\n")
        
    dprint("\n----------------------------------")
    if status == OptimizationStatus.OPTIMAL:
        dprint("Status de la résolution: OPTIMAL")
    elif status == OptimizationStatus.FEASIBLE:
        dprint("Status de la résolution: TEMPS LIMITE et SOLUTION REALISABLE CALCULEE")
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        dprint("Status de la résolution: TEMPS LIMITE et AUCUNE SOLUTION CALCULEE")
    elif status == OptimizationStatus.INFEASIBLE or status == OptimizationStatus.INT_INFEASIBLE:
        dprint("Status de la résolution: IRREALISABLE")
    elif status == OptimizationStatus.UNBOUNDED:
        dprint("Status de la résolution: NON BORNE")
    print("----------------------------------")

    # Si le modèle a été résolu à l'optimalité ou si une solution a été trouvée dans le temps limite accordé
    if model1.num_solutions>0:
        dprint("Solution calculée")
        dprint("-> Valeur de la fonction objectif de la solution calculée : ",  model1.objective_value)
        # ne pas oublier d'arrondir si le coût doit être entier
        dprint("-> Meilleure borne inférieure sur la valeur de la fonction objectif = ", model1.objective_bound)
        for i in range(n):
            # x[i].x pour accéder à la valeur de la variable !
            dprint("x[", i,"] vaut ", x[i].x)
            dprint("R vaut ", R.x)
    else:
        dprint("Pas de solution calculée")
    dprint("----------------------------------\n")


----------------------------------
Status de la résolution: IRREALISABLE
----------------------------------
Pas de solution calculée
----------------------------------



In [ ]:
## TODO:
# trouver pourquoi c'est irréalisable
# puis faire sur toutes les instanceses
# faire le deuxième modèle
# finir le rapport

## Deuxième modèle

Il est possible de modéliser le rayon de manière plus précise. Pour cela, on définit la fonction $\sigma : V \mapsto \{1, \dots, n\}$ qui associe à chaque site $i \in V$ sa position dans l'ordre croissant des distances des sites au centre et $\sigma^{-1}(k)$ la fonction inverse qui renvoie le site en position $k$ dans cet ordre\
($0 = d_r = d_{\sigma^{-1}(1)} \leq d_{\sigma^{-1}(2)} \leq \dots \leq d_{\sigma^{-1}(n)}$). Pour tout site $i \in V \setminus \{r\}$ différent du centre $r$, on pose $\Delta_i = d_i - d_{\sigma^{-1}(\sigma(i)-1)}$ la différence de distance entre le site $i$ et son prédécesseur dans l'ordre défini pas $\sigma$.

Considérez les variables suivantes

- $x_i = \left\{ \begin{array}{l}
1 \text{ si le site } i \in V \text{ est sélectionné,}\\
0 \text{ sinon.}
\end{array}\right.$      
- $y_i = \left\{ \begin{array}{l}
1 \text{ si le site } i \in V \text{ a une distance plus petite ou égale que le rayon du centre,}\\
0 \text{ sinon.}
\end{array}\right.$

Notez qu'un sommet peut avoir une distance plus petite que le rayon, sans nécessairement être sélectionné dans la solution. En revanche, un sommet sélectionné doit forcement avoir une distance plus petite ou égale au rayon. Notez également que le rayon d'un centre est la somme des $\Delta_i$ sur tous les sites $i$ ayant une distance plus petite ou égale au rayon.

Ecrivez et implémentez ce second modèle ci-dessous. Vous testerez ce modèle sur les instances fournies.

## Travail à rendre

Vous rendrez sur Moodle :

- un court rapport (au format pdf) présentant les modèles et les résultats obtenus ;
- le fichier `.ipynb` contenant vos modèles en python.
